In [16]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_model_optimization as tfmot

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [14]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2803 - accuracy: 0.9217 - val_loss: 0.1149 - val_accuracy: 0.9690
Epoch 2/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1063 - accuracy: 0.9690 - val_loss: 0.0769 - val_accuracy: 0.9798
Epoch 3/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0796 - accuracy: 0.9764 - val_loss: 0.0706 - val_accuracy: 0.9832
Epoch 4/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0670 - accuracy: 0.9805 - val_loss: 0.0706 - val_accuracy: 0.9817


In [17]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9768999814987183
Saved baseline model to: C:\Users\Jay\AppData\Local\Temp\tmpyedwmpf4.h5


C:\Users\Jay\AppData\Local\Temp\ipykernel_25388\237956268.py:7: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [18]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e_1 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
 _2 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d_2 (PruneLowMagnitu                                      
 de)                                                             
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n_1 (PruneLowMagnitude)                                         
                                                      

In [19]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 4s 7ms/step - loss: 0.1395 - accuracy: 0.9607 - val_loss: 0.1830 - val_accuracy: 0.9572
Epoch 2/2
422/422 [==============================] - 3s 7ms/step - loss: 0.1469 - accuracy: 0.9638 - val_loss: 0.1084 - val_accuracy: 0.9717


In [20]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9768999814987183
Pruned test accuracy: 0.9660999774932861


In [24]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\Jay\AppData\Local\Temp\ipykernel_25388\1300288761.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\Jay\AppData\Local\Temp\tmpuhlqagf4.h5


In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmp20xwe11e\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmp20xwe11e\assets


Saved pruned TFLite model to: C:\Users\Jay\AppData\Local\Temp\tmpaqhrsw4s.tflite


In [34]:
# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)


_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

# TFLite 변환 (가지치기 모델을 양자화 모델로 변환)
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
quantized_tflite_model = converter.convert()

# TFLite 모델 저장
with open("quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# 양자화 모델 평가
_, q_aware_model_accuracy = q_aware_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", q_aware_model_accuracy)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_4 (Quantize  (None, 28, 28)            3         
 Layer)                                                          
                                                                 
 quant_reshape_1 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_2 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_2 (Qua  (None, 13, 13, 12)        1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_1 (QuantizeW  (None, 2028)             

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpo_ghd4uz\assets


313/313 [==============================] - 1s 2ms/step - loss: 0.1250 - accuracy: 0.9649
양자화 모델 정확성: 0.964900016784668


성능 비교:
원본 모델 정확성: 0.9886999726295471
가지치기 모델 정확성: 0.9660999774932861
양자화 모델 정확성: 0.11349999904632568


AttributeError: 'str' object has no attribute 'to_json'

In [37]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_model_optimization as tfmot

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(file):
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the base model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the base digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=4,
    validation_split=0.1,
)

# Evaluate the baseline model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

# Save the baseline model in HDF5 format
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

# Apply pruning to the base model
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1  # 10% of training set will be used for the validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.80,
                                                             begin_step=0,
                                                             end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])

model_for_pruning.summary()

logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                      callbacks=callbacks)

# Evaluate the baseline model and pruned model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, pruned_model_accuracy = model_for_pruning.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', pruned_model_accuracy)

# Strip pruning from the pruned model
final_pruned_model = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Additional part
model_for_export = tfmot.sparsity.keras.strip_pruning(final_pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

# Quantization-aware training (QAT)
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for quantization-aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

q_aware_model.summary()

train_images_subset = train_images[0:1000]  # Subset of training data (for faster demonstration)
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

# Evaluate the baseline model and quantization-aware model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quantization-aware test accuracy:', q_aware_model_accuracy)

# Convert the quantization-aware model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model
with open("quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# Evaluate the quantized TFLite model
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Run inference on the test dataset
quantized_model_accuracy = 0
for test_image, test_label in zip(test_images, test_labels):
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    predicted_label = np.argmax(output)
    if predicted_label == test_label:
        quantized_model_accuracy += 1

quantized_model_accuracy /= len(test_images)

print("Quantized TFLite model accuracy:", quantized_model_accuracy)

# Compare model sizes
original_model_size = os.path.getsize(keras_file)
pruned_model_size = os.path.getsize(pruned_keras_file)
quantized_model_size = os.path.getsize("quantized_model.tflite")

print("Original model size:", original_model_size, "bytes")
print("Pruned model size:", pruned_model_size, "bytes")
print("Quantized TFLite model size:", quantized_model_size, "bytes")


Epoch 1/4


1688/1688 [==============================] - 6s 3ms/step - loss: 0.3281 - accuracy: 0.9066 - val_loss: 0.1383 - val_accuracy: 0.9642
Epoch 2/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1320 - accuracy: 0.9627 - val_loss: 0.0936 - val_accuracy: 0.9773
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0902 - accuracy: 0.9748 - val_loss: 0.0748 - val_accuracy: 0.9802
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0720 - accuracy: 0.9793 - val_loss: 0.0658 - val_accuracy: 0.9818
Baseline test accuracy: 0.9790999889373779
Saved baseline model to: C:\Users\Jay\AppData\Local\Temp\tmpyyxcv088.h5
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e_2 (PruneLowMagnitude)                                         
                                     

C:\Users\Jay\AppData\Local\Temp\ipykernel_25388\3009220999.py:55: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


Epoch 1/2
422/422 [==============================] - 5s 9ms/step - loss: 0.1107 - accuracy: 0.9702 - val_loss: 0.1217 - val_accuracy: 0.9708
Epoch 2/2
422/422 [==============================] - 3s 8ms/step - loss: 0.1227 - accuracy: 0.9690 - val_loss: 0.0975 - val_accuracy: 0.9730
Baseline test accuracy: 0.9714000225067139
Pruned test accuracy: 0.9714000225067139


C:\Users\Jay\AppData\Local\Temp\ipykernel_25388\3009220999.py:114: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\Jay\AppData\Local\Temp\tmpbewu45db.h5
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_5 (Quantize  (None, 28, 28)            3         
 Layer)                                                          
                                                                 
 quant_reshape_2 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_3 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_3 (Qua  (None, 13, 13, 12)        1         
 ntizeWrapperV2)                                                 
                                            

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpidtw2qdo\assets


Quantized TFLite model accuracy: 0.9711
Original model size: 98928 bytes
Pruned model size: 98928 bytes
Quantized TFLite model size: 88200 bytes
